In [6]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
notes = notes[['Category','Categorization','Word','Definition']].copy()

definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category'].isin(['Machine Learning','Visualizations'])]

#from df_processing import notes_df_to_outline_html,final_dataset_for_markdown
#d_learning_notes = final_dataset_for_markdown(notes,definitions)
#d_learning_notes

In [ ]:
# DQ Rules
# No Duplication in definition.Word

In [7]:
def final_dataset_for_markdown_v2(notes=None,
                               definitions=None,
                               export_location='/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/'):
    
    '''
    
    Function which helps to combined Notes and Definitions into a Single Combined Representation which can ultimately be used as a Learning Reference Tools.

    How this function Works:
    It takes the two sheets and attemps to Consolidate them together to make Final Dataset, generated as d_learning_notes.csv.

    Step 1: Insert Definition for Values in Column CATEGORY has a definition record in column WORD
            notes.Category = definitions.Word
    Step 2: Insert Definition for Values in Column Category where the Corresponding Note
            notes.Category = definition.Category and definition.Categorization = Definition
   Step 3: Find All Records which
   Step 4: Insert Parameter Mapping, where List Type Options Described in Definitions.
           notes.Categorization = definitions.Category and notes.Word = definitions.Categorization

    Order Follows what is in Notes. Uses filtering to rank, and places values with Categorization of Definition for each topic at the top.

    Parameters:
        notes(df): DataFrame of D Notes as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv
        definitions(df): DataFrame of D Definitions as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv
        export_location(str): Location of where to Save CSV File. If blank, no CSV is made.

    date_created:20-Dec-25
    date_last_modified: 21-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        final_dataset_for_markdown()


    ##############

    Has been tested for a Single Value - Machine Learning. Need to Validate once extending.

    ##############

    
    '''

    from data_d_dicts import links

    try:
        len(notes)
    except:
        notes = pd.read_csv(links['google_notes_csv'])

    try:
        len(definitions)
    except:
        definitions = pd.read_csv(links['google_definition_csv'])
    
    temp_def = definitions[['Category','Categorization','Word','Definition']].copy()

    # Step 1
    # Create Unique Category DF
    unique_cat_df = notes.drop_duplicates(['Category'])[['Category']]

    # Process temp_def such that 
    step1_df = unique_cat_df.merge(temp_def.drop('Category',axis=1),left_on='Category',right_on='Word',how='left')
    step1_df['Categorization'] = 'Definition'
    step1_df['Word'] = step1_df['Category'].copy()
    step1_df['Definition'] = step1_df['Definition'].fillna('Not Defined')

    # Step 2
    unique_cat_df['Categorization'] = 'Definition'
    step2_df = unique_cat_df.merge(temp_def,on=['Category','Categorization'],how='inner')

    # Step 3
    unique_cat_cat = 'd'

    # Step 4
    mod_def = definitions[['Category','Categorization','Word','Definition']].copy()
    mod_def['Definition'] = mod_def['Word'] + ": " + mod_def['Definition']
    mod_def = mod_def.drop('Word',axis=1).reset_index(drop=True)
    mod_def = mod_def.rename(columns={'Categorization':'Word','Category':'Categorization'})
    
    # Merge Notes.Categorization and Notes.Word to Definitions.Category and Definitions.Categorization
    step_4df = notes.drop_duplicates(['Categorization','Word'])[['Category','Categorization','Word']].merge(mod_def,on=['Categorization','Word'],how='inner')
    
    # Can DO ORDER AT THE END. USING THE ORIGINAL DATASET and Category MERGE IN

    final_df = pd.concat([notes,step1_df,step2_df,step_4df]).drop_duplicates()

    rank_df1 = notes.drop_duplicates('Category')[['Category']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank1"})
    rank_df2 = notes.drop_duplicates(['Category','Categorization'])[['Category','Categorization']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank2"})
    rank_df2['Rank2'] = rank_df2['Rank2'] + 1
    rank_df3 = notes.drop_duplicates(['Category','Categorization','Word']).reset_index(drop=True).reset_index().drop('Definition',axis=1).rename(columns={'index':'Rank3'})
    rank_df3['Rank3'] = rank_df3['Rank3'] + 1
    final_df = final_df.merge(rank_df1,on=['Category'],how='left').merge(rank_df2,on=['Category','Categorization'],how='left').merge(rank_df3,on=['Category','Categorization','Word'],how='left').fillna(0).sort_values(['Rank1','Rank2','Rank3']).reset_index(drop=True).drop(['Rank1','Rank2','Rank3'],axis=1)


    final_df['COUNT'] = final_df.groupby(['Category','Categorization','Word']).transform('size')
    final_df = final_df[~((final_df['Definition']=='')&(final_df['COUNT']!=1))].drop('COUNT',axis=1).reset_index(drop=True)
    # Need to Delete

    if export_location:
        final_df.to_csv(f"{export_location}d_learning_notes.csv",index=False)
    
    return final_df

d = final_dataset_for_markdown_v2(notes,definitions)  
d

#d_learning_notes = final_dataset_for_markdown(notes,definitions)


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
2,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
3,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
...,...,...,...,...
354,Mathematics,Best Linear Unbiased Estimator,Requirements,Linearity: Relationship between Independent an...
355,Mathematics,Best Linear Unbiased Estimator,Requirements,Independence.: Observations should be independ...
356,Mathematics,Best Linear Unbiased Estimator,Requirements,Homoscedasticity: Variance of the residuals sh...
357,Mathematics,Best Linear Unbiased Estimator,Requirements,Normality of Residuals: Residuals should be no...


In [134]:
def search_df_word(df,word,columns=None,case_sensitive=False,print_=True):
    '''
    
    
    '''

    # If User does not define which columns they wish to search, search all.
    if not columns:
        columns = df.columns.tolist()

    final_dict = {}
    final_df = pd.DataFrame()

    for column in columns:
        match_df = df[df[column].fillna('').str.contains(word,case=case_sensitive)]
        if len(match_df)==0:
            final_dict[column] = 'No Matches'
        else:
            final_word = ""
            for word in match_df[column]:
                final_word += word + ","
        
            final_dict[column] = final_word
            final_df = pd.concat([final_df,match_df])

    if print_:
        try:
            print(final_df)
        except:
            pass
            
    return final_df, final_dict

#final_df,final_dict = search_df_word(d_learning_notes,'regularization')
final_df,final_dict = search_df_word(d_learning_notes,'lasso')

    

### Examples of How Information is Included

#### Example Step 1A: Machine Learning

In [17]:
def step1a_example(df=d_learning_notes,
                   notes=notes,
                   definitions=definitions):
    
    print(f'Notes DataFrame:\nIncludes All Items with a record in Category')
    print(f'As Illustrated below by a Value Count\n')
    print(df['Category'].value_counts())

    values = df.drop_duplicates("Category")["Category"].tolist()

    print('Definition DataFrame:\nLooks for Records with the Category

    

    print(f'\nFinal D Learning Notes')
    display(d_learning_notes[d_learning_notes['Word'].isin(values)])

step1a_example()

Notes DataFrame:
Includes All Items with a record in Category
As Illustrated below by a Value Count

Category
Machine Learning    373
Mathematics           7
Name: count, dtype: int64

Final D Learning Notes


,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
373,Mathematics,Definition,Mathematics,Not Defined


In [ ]:
def step1b_example(word,
                   df=d_learning_notes,
                   notes=notes,
                   definitions=definitions):
    


STEP1A_EXAMPLE()

In [149]:
# Note there is No Explicit Record in this Sheet, simply the fact that it has a Value in Category
notes[notes['Category']=='Machine Learning']['Category'].value_counts()

Category
Machine Learning    115
Name: count, dtype: int64

In [130]:
# Record in DEFINITIONS Sheet
definitions[definitions['Word']=='Machine Learning'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
7,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...


In [148]:
# Record in NOTES Sheet
d_learning_notes[d_learning_notes['Word']=='Machine Learning']

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...


#### Example Step 1B: Bias - Variance Trade Off

In [146]:
definitions[definitions['Word'].str.contains('bias -',case=False)][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...


##### Example 2: Linear Regression

In [24]:
# Linear Regression is Stored as a Model, Algorithm
definitions[definitions['Word']=='Linear Regression'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
109,Model,Algorithm,Linear Regression,Models the relationship between an independent...


In [26]:
# It is included because there is a Corresponding Record in Notes for Model and Algorithm
notes[(notes['Categorization']=='Model')&(notes['Word']=='Algorithm')]

,Category,Categorization,Word,Notes
47,Machine Learning,Model,Algorithm,


In [20]:
# It appears as a record in Notes, Along with All other Models
d_learning_notes[d_learning_notes['Notes'].fillna("").str.contains('Linear Regression',case=False)]

,Category,Categorization,Word,Notes
164,Machine Learning,Model,Algorithm,Linear Regression: Models the relationship bet...


##### Example 2: Activation Function

In [12]:
# It is stored as a Part of Training, and a Definition, which is something we need to understand Training.
definitions[definitions['Category']=='Training'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
254,Training,Definition,Activation Function,A mathematical function applied to a neuron’s ...
255,Training,Definition,Loss Function,A loss function is a mathematical function tha...
256,Training,Definition,Optimizer,Optimization algorithm that updates a model’s ...


In [29]:
# It is included because there is a Corresponding Categorization Training. 
# Note this is a Unique Example, because it is the Special Type DEFINITION, which are automatically Included and DO NOT need to be included
notes[(notes['Categorization']=='Training')]

,Category,Categorization,Word,Notes
51,Machine Learning,Training,Goal,
52,Machine Learning,Training,Approach,
53,Machine Learning,Training,Important to Remember,
54,Machine Learning,Training,Lesson Learnt,
55,Machine Learning,Training,Algorithm,
56,Machine Learning,Training,Function,
57,Machine Learning,Training,Procedure,
58,Machine Learning,Training,TBD,


In [28]:
# It appears as a record in Notes, Along with All other Models
d_learning_notes[d_learning_notes['Notes'].fillna("").str.contains('Activation Function:',case=False)]

,Category,Categorization,Word,Notes
295,Machine Learning,Training,Definition,Activation Function: A mathematical function a...


['Problem Definition',
 'Data Collection',
 'Data Preperation',
 'Feature Engineering',
 'Feature Selection',
 'Model',
 'Training',
 'Hyperparameter Tuning',
 'Validation',
 'Evaluation',
 'Bias, Fairness and Ethics',
 'Model Summarization/ Model Interpretability',
 'Deployment',
 'Monitoring']